In [2]:
#=
\equiv+TAB -> ≡ -> ===
\nequiv+TAB -> ≢  -> !===
\ne+TAB -> ≠ -> !==
\ge+TAB -> ≥ -> >=
\le+TAB -> ≤ -> <=
\in+TAB -> ∈ -> in
\notin+TAB -> ∉ -> notin
\euler+TAB -> ℯ
\pi+TAB -> π
=#

In [3]:
#=
Objetos tipo rango
range₁ = Base.OneTo(10)
typeof(range₁)
for i in range₁
    display(i)
end
=#

In [4]:
using LinearAlgebra, Symbolics, LaTeXStrings, Latexify, SymbolicUtils

In [5]:
SymbolicUtils.show_simplified[] = true

true

In [6]:
@variables r::Real s::Real h_1(r,s) h_2(r,s) h_3(r,s) h_4(r,s) x(r,s) y(r,s)

8-element Vector{Num}:
         r
         s
 h_1(r, s)
 h_2(r, s)
 h_3(r, s)
 h_4(r, s)
   x(r, s)
   y(r, s)

In [7]:
h_1 = 1//4*(1+r)*(1+s)
h_2 = 1//4*(1-r)*(1+s)
h_3 = 1//4*(1-r)*(1-s)
h_4 = 1//4*(1+r)*(1-s)
H = [h_1 h_2 h_3 h_4]

1×4 Matrix{Num}:
 (1//4)*(1 + r)*(1 + s)  (1//4)*(1 - r)*(1 + s)  …  (1//4)*(1 + r)*(1 - s)

In [8]:
Dr = Differential(r)
Ds = Differential(s)

(::Differential) (generic function with 3 methods)

In [9]:
∇H = [transpose([expand_derivatives(Dr(H[i])) for i in 1:4]); transpose([expand_derivatives(Ds(H[i])) for i in 1:4])]

2×4 Matrix{Num}:
 (1//4)*(1 + s)  (-1//4)*(1 + s)  (-1//4)*(1 - s)   (1//4)*(1 - s)
 (1//4)*(1 + r)   (1//4)*(1 - r)  (-1//4)*(1 - r)  (-1//4)*(1 + r)

In [10]:
NodalCoords =0.01*[5.5 4; 0.5 4; 0 0; 6 0]

4×2 Matrix{Float64}:
 0.055  0.04
 0.005  0.04
 0.0    0.0
 0.06   0.0

In [11]:
#x=h₁*x₁+h₂*x₂+h₃*x₃+h₄*x₄ 
#x = expand((H*NodalCoords[:,1])[1])
#display(x)
x = 0.03+0.0275*r-0.0025*r*s

0.03 + 0.0275r - 0.0025r*s

In [12]:
# y(r,s)=h₁Y₁+h₂y₂+h₃y₃+h₄y₄ 
y= expand((H*NodalCoords[:,2])[1])

0.02 + 0.02s

In [13]:
J =expand.(∇H*NodalCoords)
#J = expand.(Grad_H*NodalCoords)
J[2,1] = -0.0025*r
display(J)

2×2 Matrix{Num}:
  0.0275 - 0.0025s  0.0
 -0.0025r           0.02

In [14]:
println(latexify(J))

\begin{equation}
\left[
\begin{array}{cc}
0.0275 - 0.0025 s & 0 \\
 - 0.0025 r & 0.02 \\
\end{array}
\right]
\end{equation}



In [15]:
J_inv = expand.(inv(J))

2×2 Matrix{Num}:
  0.02 / (0.00055 - 5.0e-5s)      -0.0
 (0.0025r) / (0.00055 - 5.0e-5s)  50.0

In [16]:
dh1dx = expand_derivatives(Dr(h_1))*J_inv[1,1] + expand_derivatives(Ds(h_1))*J_inv[1,2]
dh2dx = expand_derivatives(Dr(h_2))*J_inv[1,1] + expand_derivatives(Ds(h_2))*J_inv[1,2]
dh3dx = expand_derivatives(Dr(h_3))*J_inv[1,1] + expand_derivatives(Ds(h_3))*J_inv[1,2]
dh4dx = expand_derivatives(Dr(h_4))*J_inv[1,1] + expand_derivatives(Ds(h_4))*J_inv[1,2]

(0.005(1 - s)) / (0.00055 - 4.9999999999999975e-5s)

In [17]:
dh1dy = expand_derivatives(Dr(h_1))*J_inv[2,1] + expand_derivatives(Ds(h_1))*J_inv[2,2]
dh2dy = expand_derivatives(Dr(h_2))*J_inv[2,1] + expand_derivatives(Ds(h_2))*J_inv[2,2]
dh3dy = expand_derivatives(Dr(h_3))*J_inv[2,1] + expand_derivatives(Ds(h_3))*J_inv[2,2]
dh4dy = expand_derivatives(Dr(h_4))*J_inv[2,1] + expand_derivatives(Ds(h_4))*J_inv[2,2]

(-0.006875 - 0.00625r + 0.0006249999999999997s - 3.2526065174565133e-19r*s) / (0.00055 - 4.9999999999999975e-5s)

In [18]:
B = [dh1dx 0 dh2dx 0 dh3dx 0 dh4dx 0;
    0 dh1dy 0 dh2dy 0 dh3dy 0 dh4dy;
    dh1dy dh1dx dh2dy dh2dx dh3dy dh3dx dh4dy dh4dx]

3×8 Matrix{Num}:
 (0.005(1 + s)) / (0.00055 - 5.0e-5s)                                     …    0
  0                                                                          (-0.006875 - 0.00625r + 0.000625s - 3.25261e-19r*s) / (0.00055 - 5.0e-5s)
 (0.006875 + 0.0075r - 0.000625s + 3.25261e-19r*s) / (0.00055 - 5.0e-5s)      (0.005(1 - s)) / (0.00055 - 5.0e-5s)

In [19]:
println(latexify(B))

\begin{equation}
\left[
\begin{array}{cccccccc}
\frac{0.005 \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 & \frac{ - 0.005 \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 & \frac{ - 0.005 \left( 1 - s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 & \frac{0.005 \left( 1 - s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 \\
0 & 12.5 \left( 1 + r \right) + \frac{0.000625 r \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 & 12.5 \left( 1 - r \right) + \frac{ - 0.000625 r \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 &  - 12.5 \left( 1 - r \right) + \frac{ - 0.000625 r \left( 1 - s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 0 &  - 12.5 \left( 1 + r \right) + \frac{0.000625 r \left( 1 - s \right)}{0.00055 - 5 \cdot 10^{-5} s} \\
12.5 \left( 1 + r \right) + \frac{0.000625 r \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & \frac{0.005 \left( 1 + s \right)}{0.00055 - 5 \cdot 10^{-5} s} & 12.5 \left( 1 - r \right) + \frac{ - 0.000625 r \left( 1 + s \right)}{0.00055 - 5 \cd

In [20]:
U = [0.022, 0.001, 0.023, 0.001, 0.02, 0.001, 0.025, 0.001]

8-element Vector{Float64}:
 0.022
 0.001
 0.023
 0.001
 0.02
 0.001
 0.025
 0.001

In [21]:
#E = simplify.(expand.(B*U))
E = expand.(B*U)

3-element Vector{Num}:
 0
 0
 0

In [22]:
E = [(2-3*s)/(55-5s), 0, (9.7*r-37.5*r*s)/(55-5s)]

3-element Vector{Num}:
 (2 - 3s) / (55 - 5s)
                    0
                   (9.7r - 37.5r*s) / (55 - 5s)

In [23]:
-0.0001+0.00011-0.000115+0.000125
#0.0001+0.00011-0.000115-0.000125
0.00055-5e-5*s
#55e-5

0.00055 - 5.0e-5s

In [24]:
expand(-0.075*r+0.02*(-0.000625*r+0.000625*r*s)+0.022*(0.000625*r+0.000625*r*s)+0.023*(-0.000625*r-0.000625*r*s)+0.025*(0.000625*r-0.000625*r*s))

-0.07499749999999998r - 3.7499999999999997e-6r*s

In [25]:
expand(E[1])
println(latexify(E))
#expand(−0.0001*(1−s)+0.00011*(1+s)−0.000115*(1+s)+0.000125*(1−s))

\begin{equation}
\left[
\begin{array}{c}
\frac{2 - 3 s}{55 - 5 s} \\
0 \\
\frac{9.7 r - 37.5 r s}{55 - 5 s} \\
\end{array}
\right]
\end{equation}



In [26]:
expand(E[2])
#println(latexify(E[2]))

0

In [27]:
expand(E[3])

(9.7r - 37.5r*s) / (55 - 5s)

In [28]:
E1 = 2.1e6
v = 0.2
C = (E1*(1-v)/((1+v)*(1-2*v)))*[1 v/(1-v) 0; v/(1-v) 1 0; 0 0 (1-2*v)/(2-2*v)]

3×3 Matrix{Float64}:
 2.33333e6  5.83333e5       0.0
 5.83333e5  2.33333e6       0.0
 0.0        0.0        875000.0

In [29]:
esf = C*E

3-element Vector{Num}:
      (2.3333333333333335e6(2 - 3s)) / (55 - 5s)
 (583333.3333333334(2 - 3s)) / (55 - 5s)
 (874999.9999999999(9.7r - 37.5r*s)) / (55 - 5s)

In [30]:
esf[1]

(2.3333333333333335e6(2 - 3s)) / (55 - 5s)

In [31]:
# x=3cm., y=2cm.
Ea = substitute(E, Dict([r => 0, s => 0]))
#round(Ea; digits = 6)

3-element Vector{Num}:
 0.03636363636363636
 0
 0.0

In [32]:
sigma_a = C*Ea

3-element Vector{Num}:
 84848.48484848485
 21212.121212121212
     0.0

In [33]:
sigma_zz_a = v*(sigma_a[1]+sigma_a[2])

21212.121212121216

In [34]:
Eb = substitute(E, Dict([r => -1, s => -1]))

3-element Vector{Num}:
  0.08333333333333333
  0
 -0.7866666666666667

In [35]:
sigma_b = C*Eb

3-element Vector{Num}:
  194444.44444444444
   48611.11111111111
 -688333.3333333333

In [36]:
sigma_zz_b = v*(sigma_b[1]+sigma_b[2])

48611.11111111112

In [37]:
Ec = substitute(E, Dict([r => 8/11, s => 0]))

3-element Vector{Num}:
 0.03636363636363636
 0
 0.12826446280991735

In [38]:
sigma_c = C*Ec

3-element Vector{Num}:
  84848.48484848485
  21212.121212121212
 112231.40495867767

In [39]:
sigma_zz_c = v*(sigma_c[1]+sigma_c[2])

21212.121212121216